## Getting data from Wikidata for cybersecurity annotattion and pipelines
 * get malware names, #sites, and alias (#sites is the number of wikipedeia sites with a page on the malware and is a quick way to estimate significance)
 * get english text from wikipedia via DBpedia for a malware instance
 * do these for other concepts, like operating systems, exploits, 

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

### Code and variables for accessing Wikidata and DBpedia via SPARQL

In [2]:
# default endpoint and user id for querying wikidata with sparal
WIKIDATA_ENDPOINT = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
DBPEDIA_ENDPOINT =  "http://dbpedia.org/sparql"
USER_AGENT = "Tim.Finin_cyber"

def query_endpoint(query, endpoint=WIKIDATA_ENDPOINT):
    """ send query to endpoint and return response as JSON """
    sparql = SPARQLWrapper(endpoint, agent=USER_AGENT)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    return sparql.query().convert()

### Query to get label, # sites, and aliases for instances of a class

In [3]:
la_query = """
SELECT DISTINCT ?label ?sites (group_concat(distinct ?alias; separator='|') as ?aliases)
WHERE {{
  ?x wdt:P31/wdt:P279* wd:{QID};
     rdfs:label ?label;
     wikibase:sitelinks ?sites.
  FILTER (?sites >= {MIN})
  FILTER(lang(?label) = "en").
  OPTIONAL {{?x skos:altLabel ?alias. FILTER(lang(?alias)="en") }}
  }}
GROUP BY ?label ?sites
ORDER BY DESC(?sites)
"""

In [32]:
subclass_query = """
SELECT DISTINCT ?label ?sites (group_concat(distinct ?alias; separator='|') as ?aliases)
WHERE {{
  ?x wdt:P279+ wd:{QID}.
  # ?x must have either an instance or a subclass
  FILTER (EXISTS {{?y wdt:P31|wdt:P279 ?x}} )
  ?x rdfs:label ?label;
     wikibase:sitelinks ?sites.
  FILTER (?sites >= {MIN})
  FILTER(lang(?label) = "en").
  OPTIONAL {{?x skos:altLabel ?alias. FILTER(lang(?alias)="en") }}
  }}
GROUP BY ?label ?sites
ORDER BY DESC(?sites)
"""

### Given the QID of a class, return or print  list of the name, # sites, and aliases of each instance 

 * sites is the number of different Wikipedia sites that have a page for the item.  It's a simple way to estimate the item's importance
 * for some items, we may want to limit the results to those with a minimum number of Wikipedia sites

In [40]:
def get_instance_names(id, min_sites=0):
    """ given a class qid, returns list of name, # sites, and aliases of each instance that has 
        at least min_sites Wikipedia sites """
    q = la_query.format(QID=id, MIN=min_sites)
    results = query_endpoint(q)
    answers = []    
    for rs in  results["results"]["bindings"]:
        label = rs['label']['value'] if 'label' in rs else ''
        aliases = rs['aliases']['value'].split('|') if 'aliases' in rs else []
        aliases = [] if aliases == [''] else aliases
        sites = int(rs['sites']['value'])
        answers.append([label, sites] + aliases)
    return answers

def get_subclass_names(id, min_sites=0):
    """ given a class qid, returns list of name, # sites, and aliases of each instance that has 
        at least min_sites Wikipedia sites """
    q = subclass_query.format(QID=id, MIN=min_sites)
    results = query_endpoint(q)
    answers = []    
    for rs in  results["results"]["bindings"]:
        label = rs['label']['value'] if 'label' in rs else ''
        aliases = rs['aliases']['value'].split('|') if 'aliases' in rs else []
        aliases = [] if aliases == [''] else aliases
        sites = int(rs['sites']['value'])
        answers.append([label, sites] + aliases)
    return answers

def get_instance_qids(QID, min_sites=0):
    """ given a class qid, returns list of qids of each instance that has 
        at least min_sites Wikipedia sites """
    q = f"SELECT DISTINCT ?xid WHERE {{ ?xid wdt:P31/wdt:P279* wd:{QID}; wikibase:sitelinks ?n. FILTER (?n >= {min_sites}) }}"
    results = query_endpoint(q)
    answer = []    
    for rs in  results["results"]["bindings"]:
        answer.append(rs['xid']['value'].split('/')[-1])
    return answer

def write_instance_names(id, min_sites=1, fileName=''):
    """ write tab-seperated data"""
    if not fileName:
        fileName = id + '.tsv'
    with open(fileName, 'w') as out:
        for data in get_instance_names(id):
            name = data[0]
            sites = data[1]
            aliases = '\t'.join(data[2:])
            out.write(f"{name}\t{sites}\t{aliases}\n")
    return True

def write_subclass_names(id, min_sites=1, fileName=''):
    """ write tab-seperated data"""
    if not fileName:
        fileName = id + '.tsv'
    with open(fileName, 'w') as out:
        for data in get_subclass_names(id):
            name = data[0]
            sites = data[1]
            aliases = '\t'.join(data[2:])
            out.write(f"{name}\t{sites}\t{aliases}\n")
    return True

### Examples of names we can collect for some concepts
 * we might use these to create EntityRuler patterns for cubersecurity NER

In [27]:
malware = "Q14001"
get_instance_names(malware, 20) 

[['Stuxnet', 37, 'RootkitTmphider', 'W32.Stuxnet', 'W32.Temphid'],
 ['ILOVEYOU', 36, 'I love you', 'I Love You', 'Love Letter', 'Loveletter'],
 ['Conficker', 31, 'Kido', 'Downadup', 'Downup'],
 ['Morris worm', 31, 'Internet worm of November 2, 1988'],
 ['Pegasus', 29, 'lenguis'],
 ['Petya', 27, 'GoldenEye', 'NotPetya', 'Petya.2017'],
 ['Melissa', 26, 'W97M.Melissa.A'],
 ['Brain', 24, '(c)Brain'],
 ['CryptoLocker', 22],
 ['CIH', 21, 'Chernobyl', 'Spacefiller'],
 ['Flame', 20, 'Flamer', 'Da Flame', 'sKyWIper', 'Skywiper'],
 ['Code Red', 20]]

In [8]:
write_instance_names("Q14001", 10, "malware.tsv")

True

In [9]:
# Q9135 = operating_system
write_instance_names("Q9135", 30, "os.tsv")

True

In [10]:
# exploit (Q11287)
write_instance_names("Q11287", 1, "exploits.tsv")

True

In [11]:
# threat actor (Q28402245)
write_instance_names("Q28402245", 1, "threat_actor.tsv")
# hacker group
write_instance_names("Q2801262", 1, "hacker_group.tsv")

True

### Get or write subclass names of a concept

In [38]:
malware = "Q14001"
get_subclass_names(malware, 10) 

[['computer virus', 103, 'virus', 'Malware'],
 ['Trojan horse', 63, 'Trojan Horse', 'Trojan'],
 ['computer worm', 55, 'worm', 'worms', 'network worm'],
 ['ransomware', 55, 'denial-of-access attack', 'ransom software'],
 ['spyware', 53, 'spying software'],
 ['rootkit', 46],
 ['adware', 43],
 ['keystroke logging',
  41,
  'keylogger',
  'keyboard capturing',
  'keyboard logger',
  'keylogging',
  'keystroke logger'],
 ['Macro virus', 23],
 ['scareware', 17],
 ['mobile virus', 14],
 ['rogue security software',
  13,
  'fake antivirus',
  'false antivirus',
  'fraudware',
  'rogue antivirus',
  'rogue AV',
  'Rogue security software',
  'Rouge antivirus']]

### Write file for maleware types (subclasses of Malware)

In [41]:
write_subclass_names(malware, 0, 'malware_types.tsv') 

True

### Get DBpedia abstracts for class instances
 * query DBpedia to get abstract of cybersecurity class instances that has a minimum number of sites
 * **write_instance_abstracts** writes one abstract per line to a file


In [15]:
def write_instance_abstracts(id, min_sites=0, fileName=''):
    """ write DBpedia abstract for each class instance to filename"""
    if not fileName:
        fileName = id + '_abstracts.txt'
    with open(fileName, 'w') as out:
        for qid in get_instance_qids(id, min_sites=min_sites):
            text = get_instance_abstract(qid)
            out.write(text)
            out.write('\n')
            
def get_instance_abstract(qid):
    """ Get en DBpedia abstract from wikidata qid, returning '' when not available """
    name = get_en_wikipedia_name(qid)
    if not name: 
        return ''  # exit with '' if no name
    name = encode_string(name)
    query = f'select ?text where {{dbr:{name} dbo:abstract ?text. FILTER(lang(?text)="en")}}'
    results = query_endpoint(query, DBPEDIA_ENDPOINT)
    if results["results"]["bindings"]:
        return results["results"]["bindings"][0]['text']['value']
    else:
        return ''

def encode_string(text):
    """ prefix some chars with a backslash for using in a sparql query """
    for ch in """(),'/@""" :
        if ch in text:
            text = text.replace(ch,"\\"+ch)
    if text.endswith('.'):
        text = text[:-1] + '\.'
    return text

def get_en_wikipedia_name(qid):
    """ Given a wikidata QID, get its en Wikipedia name if it has one, else '' """
    query = f'SELECT ?name {{?art schema:about wd:{qid}; schema:inLanguage "en"; schema:name ?name; schema:isPartOf <https://en.wikipedia.org/>.}} LIMIT 1'
    results = query_endpoint(query, WIKIDATA_ENDPOINT)
    if results["results"]["bindings"]:
        return results["results"]["bindings"][0]['name']['value'].replace(' ', '_')
    else:
        return ''
    
def write_instance_abstracts(id, min_sites=0, fileName=''):
    """ write DBpedia abstract for each class instance to filename"""
    if not fileName:
        fileName = id + '_abstracts.txt'
    with open(fileName, 'w') as out:
        for qid in get_instance_qids(id, min_sites=min_sites):
            text = get_instance_abstract(qid)
            out.write(text)
            out.write('\n')
    return True


### Examples getting text abstracts for malware and threat actors

In [ ]:
# example -- get malware instance text for items with >= 4 sites
write_instance_abstracts(malware, 4, 'malware_abstracts.txt')

In [ ]:
# example -- get threat actor instance text for items with >= 4 sites
write_instance_abstracts("Q28402245", 4, 'threat_actors.txt')

### *fin* 